Локальная Валидация (Last Week Cross-Validation)


Данные в `train.parquet` занимают 4 недели. 
- Обучающая выборка (`train_cv`): первые 3 недели.
- Тестовая выборка (`test_cv`): 4-я неделя. 


In [ ]:
import polars as pl
import os
import gc

TRAIN_DIR = './data_parquet/train/'
files = [os.path.join(TRAIN_DIR, f) for f in os.listdir(TRAIN_DIR) if f.endswith('.parquet')]

lazy_df = pl.scan_parquet(TRAIN_DIR + '*.parquet')
max_ts = lazy_df.select(pl.max('ts')).collect().item()

print(f"Максимальный timestamp в данных: {max_ts}")

Максимальный timestamp в данных: 1661723999984


In [ ]:
days_7 = 7 * 24 * 60 * 60 * 1000 
split_ts = max_ts - days_7

print(f"Граница 3-й и 4-й недели (split_ts): {split_ts}")

Граница 3-й и 4-й недели (split_ts): 1661119199984



1. **Тренировочная выборка (`train_cv`)**: Все сессии, которые закончились до начала 4-й недели.
2. **Валидационная выборка**: Сессии, которые имели активность на 4-й неделе. 
   - Возьмем каждую такую сессию и отрежем последние 30% событий.
   - Первые 70% пойдут в **`test_cv`** .
   - Последние 30% пойдут в **`labels_cv`** .


In [ ]:
import os
out_train = './data_parquet/train_cv/'
out_test = './data_parquet/test_cv/'
out_labels = './data_parquet/labels_cv/'
os.makedirs(out_train, exist_ok=True)
os.makedirs(out_test, exist_ok=True)
os.makedirs(out_labels, exist_ok=True)

df = pl.read_parquet('./data_parquet/train/001.parquet')

last_ts_per_session = df.group_by('session').agg(pl.max('ts').alias('max_ts'))
valid_sessions = last_ts_per_session.filter(pl.col('max_ts') >= split_ts)['session']

df_train_cv = df.filter(~pl.col('session').is_in(valid_sessions.to_list()))

df_valid_full = df.filter(pl.col('session').is_in(valid_sessions.to_list()))

df_valid_full = df_valid_full.sort(['session', 'ts'])

df_valid_full = df_valid_full.with_columns([
    pl.col('ts').cum_count().over('session').alias('row_num'),
    pl.col('ts').count().over('session').alias('total_rows')
])

df_valid_full = df_valid_full.filter(pl.col('total_rows') >= 2)

df_test_cv = df_valid_full.filter(pl.col('row_num') <= (pl.col('total_rows') * 0.7))
df_labels_cv = df_valid_full.filter(pl.col('row_num') > (pl.col('total_rows') * 0.7))

df_test_cv = df_test_cv.drop(['row_num', 'total_rows'])
df_labels_cv = df_labels_cv.drop(['row_num', 'total_rows'])

print(f"Изначально строк в чанке: {df.shape[0]}")
print(f"Строк в обучающей выборке (train_cv): {df_train_cv.shape[0]}")
print(f"Строк в тестовой выборке (test_cv): {df_test_cv.shape[0]}")
print(f"Строк в скрытых ответах (labels_cv): {df_labels_cv.shape[0]}")

Запускаем сплит для ВСЕХ 120 файлов-чанков



In [ ]:
from tqdm.auto import tqdm

train_files = sorted(os.listdir(TRAIN_DIR))
print(f"Найдено {len(train_files)} файлов. Начинаем обработку...")

for file_name in tqdm(train_files):
    file_path = os.path.join(TRAIN_DIR, file_name)
    df = pl.read_parquet(file_path)
    
    last_ts_per_session = df.group_by('session').agg(pl.max('ts').alias('max_ts'))
    valid_sessions = last_ts_per_session.filter(pl.col('max_ts') >= split_ts)['session']
    
    df_train_cv = df.filter(~pl.col('session').is_in(valid_sessions.to_list()))
    
    df_valid_full = df.filter(pl.col('session').is_in(valid_sessions.to_list()))
    
    if len(df_valid_full) > 0:
        df_valid_full = df_valid_full.sort(['session', 'ts'])
        df_valid_full = df_valid_full.with_columns([
            pl.col('ts').cum_count().over('session').alias('row_num'),
            pl.col('ts').count().over('session').alias('total_rows')
        ])
        df_valid_full = df_valid_full.filter(pl.col('total_rows') >= 2)
        
        df_test_cv = df_valid_full.filter(pl.col('row_num') <= (pl.col('total_rows') * 0.7)).drop(['row_num', 'total_rows'])
        df_labels_cv = df_valid_full.filter(pl.col('row_num') > (pl.col('total_rows') * 0.7)).drop(['row_num', 'total_rows'])
    else:
        df_test_cv = pl.DataFrame(schema=df.schema)
        df_labels_cv = pl.DataFrame(schema=df.schema)
    
    df_train_cv.write_parquet(os.path.join(out_train, file_name))
    if len(df_test_cv) > 0:
        df_test_cv.write_parquet(os.path.join(out_test, file_name))
    if len(df_labels_cv) > 0:
        df_labels_cv.write_parquet(os.path.join(out_labels, file_name))

print("Валидационный сплит успешно создан !")

Найдено 2167 файлов. Начинаем обработку...


  0%|          | 0/2167 [00:00<?, ?it/s]

! Валидационный сплит успешно создан !
